In [1]:
#Paquetes necesarios
import cv2
import numpy as np
from deepface import DeepFace
from PIL import Image

In [2]:
# Función para superponer imágenes con canal alfa
def overlay_image_alpha(img, overlay, position, alpha_mask):
    x, y = position
    h, w = overlay.shape[:2]

    # Recorte de las coordenadas para asegurar que no se salga del frame
    if x < 0:
        overlay = overlay[:, -x:]
        alpha_mask = alpha_mask[:, -x:]
        w += x
        x = 0
    if y < 0:
        overlay = overlay[-y:, :]
        alpha_mask = alpha_mask[-y:, :]
        h += y
        y = 0
    if x + w > img.shape[1]:
        overlay = overlay[:, :img.shape[1] - x]
        alpha_mask = alpha_mask[:, :img.shape[1] - x]
        w = img.shape[1] - x
    if y + h > img.shape[0]:
        overlay = overlay[:img.shape[0] - y, :]
        alpha_mask = alpha_mask[:img.shape[0] - y, :]
        h = img.shape[0] - y

    # Crear un fondo y sobreponer con la máscara alfa
    img_crop = img[y:y+h, x:x+w]
    img_crop[:] = img_crop * (1 - alpha_mask[:, :, np.newaxis] / 255.0) + overlay * (alpha_mask[:, :, np.newaxis] / 255.0)

    return img

# Función para cargar fotogramas de un GIF
def load_gif_frames(gif_path):
    pil_gif = Image.open(gif_path)
    frames = []
    try:
        while True:
            frame = pil_gif.convert("RGBA")
            frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGBA2BGRA)
            frames.append(frame)
            pil_gif.seek(pil_gif.tell() + 1)
    except EOFError:
        pass
    return frames


In [ ]:
# Ruta de los gráficos
hair = cv2.imread('pelo_super_saiyan.png', cv2.IMREAD_UNCHANGED)
aura_frames = load_gif_frames('aura.gif')
scouter = cv2.imread('scouter.png', cv2.IMREAD_UNCHANGED)
aura_frame_count = len(aura_frames)
aura_index = 0

# Opciones del detector
detectors = ['opencv', 'mtcnn', 'retinaface', 'ssd']
detector_idx = 0

# Captura de video en tiempo real
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Detección de rostros
        faces = DeepFace.extract_faces(frame, detector_backend='mtcnn')

        for face in faces:
            # Usar las coordenadas y tamaños directamente
            x, y, w, h = face['facial_area']['x'], face['facial_area']['y'], face['facial_area']['w'], face['facial_area']['h']

            # Análisis de emoción
            face_img = frame[y:y+h, x:x+w]
            analysis = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
            emotion = analysis[0]['dominant_emotion']

            # Activar efectos si la emoción es enfado
            if emotion == 'angry':
                hair_resized = cv2.resize(hair, (w * 3, h * 2))
                hair_alpha = hair_resized[:, :, 3]
                aura_frame = aura_frames[aura_index]
                aura_resized = cv2.resize(aura_frame, (frame.shape[1] * 2, frame.shape[0] * 2))
                aura_alpha = aura_resized[:, :, 3]
                
                # Posicionar scouter en el ojo izquierdo si está disponible
                if face['facial_area']['right_eye'] is not None:
                    print("right_eye")
                    scouter_resized = cv2.resize(scouter, (int(w * 0.6), int(h * 0.3)))
                    scouter_alpha = scouter_resized[:, :, 3]
                    eye_x = x + int(face['facial_area']['right_eye'][0] * w)
                    eye_y = y + int(face['facial_area']['right_eye'][1] * h)
                    frame = overlay_image_alpha(frame, scouter_resized[:, :, :3], (eye_x, eye_y), scouter_alpha)

                # Coordenadas para el aura y el cabello
                hair_x = x + w // 2 - hair_resized.shape[1] // 2
                hair_y = y - int(hair_resized.shape[0] * 0.7)
                aura_x = x + w // 2 - aura_resized.shape[1] // 2
                aura_y = y - frame.shape[0]

                # Superposición de efectos
                frame = overlay_image_alpha(frame, aura_resized[:, :, :3], (aura_x, aura_y), aura_alpha)
                frame = overlay_image_alpha(frame, hair_resized[:, :, :3], (hair_x, hair_y), hair_alpha)

                # Avanzar el índice de fotograma para animación
                aura_index = (aura_index + 1) % aura_frame_count

    except Exception as e:
        print("Error en detección o análisis de emociones:", e)

    cv2.putText(frame, "Detector: ssd", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.imshow("Filtro Saiyan", frame)

    # Cambiar de detector o salir
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # Salir con ESC
        break

cap.release()
cv2.destroyAllWindows()

right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
right_eye
